<a href="https://colab.research.google.com/github/hy05sprsea/KIKAGAKU-Tutorial/blob/master/%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E8%AA%BF%E6%95%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ハイパーパラメータの調整方法**

**手動での調整**

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()

In [3]:
t = dataset.target
x = dataset.data

x.shape, t.shape

((569, 30), (569,))

In [4]:
# 与えられたデータを「テスト用データセット：その他 ＝ 20 ： 80 」に分割
from sklearn.model_selection import train_test_split
x_train_val, x_test, t_train_val, t_test = train_test_split(x, t, test_size=0.2, random_state=0)

In [7]:
# 検証用データセット：学習用データセット＝ 30 ： 70
x_train, x_val, t_train, t_val = train_test_split(x_train_val, t_train_val, test_size=0.3, random_state=1)

In [8]:
x_train.shape, x_val.shape, x_test.shape

((318, 30), (137, 30), (114, 30))

In [9]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=0)

In [10]:
dtree.fit(x_train, t_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [11]:
print('train score : ', dtree.score(x_train, t_train))
print('validation score : ', dtree.score(x_val, t_val))

train score :  1.0
validation score :  0.9343065693430657


In [14]:
# ハイパーパラメータを設定して、モデルの定義
dtree = DecisionTreeClassifier(max_depth=10, min_samples_split=30, random_state=0)

dtree.fit(x_train, t_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=30,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [15]:
print('train score : ', dtree.score(x_train, t_train))
print('validation score : ', dtree.score(x_val, t_val))

train score :  0.9591194968553459
validation score :  0.9124087591240876


In [16]:
# テスト用データセットに対して予測精度
print('test score : ', dtree.score(x_test, t_test))

test score :  0.9210526315789473


**グリッドサーチ**

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
# 学習に使用するアルゴリズムの定義
estimator = DecisionTreeClassifier(random_state=0)

In [19]:
# 探索するハイパーパラメータと範囲の定義
param_grid = [{
    'max_depth': [3, 20, 50],
    'min_samples_split': [3, 20, 30]
}]

In [20]:
# データセット分割数の定義
cv = 5

In [21]:
# GridSearchCV クラスを用いたモデルの定義
turned_model = GridSearchCV(estimator=estimator,
                            param_grid=param_grid,
                            cv= cv, return_train_score=True)

In [22]:
turned_model.fit(x_train_val, t_train_val)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_depth': [3, 20, 50],
                          'min_samples_split': [3, 20, 30]}],
             

In [23]:
pd.DataFrame(turned_model.cv_results_).T

,0,1,2,3,4,5,6,7,8
mean_fit_time,0.005019,0.00400138,0.0040823,0.00568376,0.00561376,0.00553923,0.00577016,0.00552521,0.00549679
std_fit_time,0.0017865,0.000243239,0.000643178,0.00062927,0.000575279,0.000668418,0.00063774,0.000660425,0.000655165
mean_score_time,0.000519037,0.00043025,0.000419331,0.000452852,0.000434589,0.000402403,0.000447941,0.000382423,0.00038271
std_score_time,0.000229027,4.59035e-05,0.000126841,8.22524e-05,2.77949e-05,3.6599e-05,4.59141e-05,2.64256e-05,2.24496e-05
param_max_depth,3,3,3,20,20,20,50,50,50
param_min_samples_split,3,20,30,3,20,30,3,20,30
params,"{'max_depth': 3, 'min_samples_split': 3}","{'max_depth': 3, 'min_samples_split': 20}","{'max_depth': 3, 'min_samples_split': 30}","{'max_depth': 20, 'min_samples_split': 3}","{'max_depth': 20, 'min_samples_split': 20}","{'max_depth': 20, 'min_samples_split': 30}","{'max_depth': 50, 'min_samples_split': 3}","{'max_depth': 50, 'min_samples_split': 20}","{'max_depth': 50, 'min_samples_split': 30}"
split0_test_score,0.912088,0.923077,0.923077,0.868132,0.934066,0.934066,0.868132,0.934066,0.934066
split1_test_score,0.901099,0.901099,0.901099,0.912088,0.901099,0.901099,0.912088,0.901099,0.901099
split2_test_score,0.89011,0.89011,0.901099,0.89011,0.89011,0.901099,0.89011,0.89011,0.901099


In [24]:
estimator = DecisionTreeClassifier(random_state=0)
cv = 5
param_grid = [{
    'max_depth': [5, 10, 15] , 
    'min_samples_split': [10, 12, 15]
}]

In [25]:
# モデルの定義
tuned_model = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=cv, return_train_score=False)

# モデルの学習
tuned_model.fit(x_train_val, t_train_val)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=0, splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid=[{'max_depth': [5, 10, 15],
                          'min_samples_split': [10, 12, 15]}],
            

In [26]:
pd.DataFrame(tuned_model.cv_results_).T

,0,1,2,3,4,5,6,7,8
mean_fit_time,0.00602732,0.00529995,0.00506997,0.00637269,0.0056778,0.00556307,0.00557089,0.00558596,0.00565748
std_fit_time,0.00165006,0.000330675,0.000204633,0.000555267,0.000676115,0.000627803,0.000634091,0.000640364,0.000742503
mean_score_time,0.000430965,0.000397921,0.000375843,0.000495672,0.000408363,0.000364542,0.000370502,0.000377464,0.000369406
std_score_time,0.000106152,6.99533e-05,2.08016e-05,0.000110772,2.89104e-05,1.77105e-05,1.36262e-05,2.69455e-05,2.04766e-05
param_max_depth,5,5,5,10,10,10,15,15,15
param_min_samples_split,10,12,15,10,12,15,10,12,15
params,"{'max_depth': 5, 'min_samples_split': 10}","{'max_depth': 5, 'min_samples_split': 12}","{'max_depth': 5, 'min_samples_split': 15}","{'max_depth': 10, 'min_samples_split': 10}","{'max_depth': 10, 'min_samples_split': 12}","{'max_depth': 10, 'min_samples_split': 15}","{'max_depth': 15, 'min_samples_split': 10}","{'max_depth': 15, 'min_samples_split': 12}","{'max_depth': 15, 'min_samples_split': 15}"
split0_test_score,0.912088,0.912088,0.912088,0.879121,0.879121,0.912088,0.879121,0.879121,0.912088
split1_test_score,0.901099,0.901099,0.945055,0.923077,0.923077,0.945055,0.923077,0.923077,0.945055
split2_test_score,0.89011,0.89011,0.89011,0.89011,0.89011,0.89011,0.89011,0.89011,0.89011


In [27]:
tuned_model.best_params_

{'max_depth': 10, 'min_samples_split': 15}

In [28]:
# 最も予測精度の高かったモデルの引き継ぎ
best_model = tuned_model.best_estimator_

# モデルの検証
print(best_model.score(x_train_val, t_train_val))
print(best_model.score(x_test, t_test))

0.978021978021978
0.9035087719298246


**ランダムサーチ**

In [29]:
from sklearn.model_selection import RandomizedSearchCV

In [30]:
estimator = DecisionTreeClassifier(random_state=0)

In [31]:
list(range(1, 10, 2))

[1, 3, 5, 7, 9]

In [43]:
# ハイパーパラメータを探索する範囲の指定
param_distributions = {
    'max_depth': list(range(5, 100, 2)),
    'min_samples_split': list(range(2, 50, 1))
}

In [44]:
# 試行回数の指定
n_iter = 100

In [45]:
cv = 5

In [46]:
# モデルの定義
tuned_model = RandomizedSearchCV(
    estimator=estimator, 
    param_distributions=param_distributions, 
    n_iter=n_iter, cv=cv, 
    random_state=0, return_train_score=False
)

In [47]:
# モデルの学習＆検証
tuned_model.fit(x_train_val, t_train_val)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=0,
             

In [48]:
pd.DataFrame(tuned_model.cv_results_).sort_values('rank_test_score').T

,46,61,75,76,37,33,91,88,38,40,41,95,98,53,55,58,60,93,64,65,67,92,78,80,81,83,84,85,86,87,89,49,99,31,17,18,30,16,10,22,...,54,90,96,68,69,28,82,14,15,1,74,8,66,51,56,63,59,43,11,42,6,47,62,77,19,29,35,3,97,39,71,50,72,21,12,20,79,70,26,0
mean_fit_time,0.00550647,0.00550003,0.00550294,0.00550351,0.00552859,0.00563793,0.00514412,0.00546722,0.00556097,0.00536509,0.00540638,0.00542212,0.00547314,0.00563836,0.005512,0.00548234,0.00545363,0.0054121,0.00539021,0.00538435,0.00549111,0.00550189,0.00536141,0.0053617,0.00533023,0.00536551,0.005372,0.00538993,0.00546546,0.00538836,0.00628247,0.00536499,0.00553732,0.00542173,0.00537462,0.00549021,0.00541101,0.00539212,0.00572209,0.00546422,...,0.00559177,0.00567598,0.00562296,0.00555639,0.00571775,0.00564613,0.00555811,0.00541172,0.00541062,0.00581298,0.00545664,0.00541196,0.00563068,0.00543022,0.00541368,0.00542736,0.00544763,0.00552802,0.00553684,0.0056149,0.00575085,0.00560966,0.00558467,0.00556827,0.00573316,0.0056066,0.0054913,0.00564513,0.00542636,0.00557961,0.00537562,0.00538321,0.0053494,0.00552921,0.0055603,0.00542612,0.00546198,0.00564442,0.0054842,0.00725703
std_fit_time,0.000630274,0.000644118,0.000598913,0.000608868,0.000614214,0.000613036,0.000231049,0.000693582,0.000519812,0.000615449,0.000651807,0.000593845,0.00062728,0.000820168,0.000649427,0.000723148,0.00058575,0.000641164,0.00063256,0.000645111,0.000619003,0.000624327,0.000602563,0.000629864,0.000621063,0.00065004,0.000624817,0.000663335,0.00068968,0.000619586,0.00147602,0.000599631,0.000764863,0.00062353,0.000629897,0.000763216,0.000617992,0.000617867,0.000495414,0.000628368,...,0.000699822,0.000675424,0.000658467,0.000639858,0.000903767,0.000687461,0.000636947,0.000622189,0.000633302,0.000505628,0.000607119,0.000639566,0.000667307,0.000624119,0.000638648,0.000620959,0.000653216,0.000631293,0.000616154,0.000626662,0.000827214,0.000651572,0.000605084,0.000624908,0.000653466,0.000615484,0.000581681,0.000650323,0.000629225,0.000626039,0.000616786,0.000602209,0.000616283,0.000564968,0.000620337,0.000571199,0.000593836,0.00062957,0.000659394,0.00261618
mean_score_time,0.000361395,0.000362873,0.000364304,0.000339413,0.000364971,0.000401974,0.000442505,0.000383854,0.000396252,0.000345802,0.000360823,0.000356436,0.000390816,0.000423336,0.000397778,0.000370836,0.00041337,0.000387144,0.000355434,0.000357199,0.000348949,0.000392246,0.000347471,0.000344324,0.00032115,0.000334072,0.000347042,0.000354815,0.000359488,0.000351191,0.000448227,0.000337934,0.000405502,0.000363302,0.000342512,0.000374365,0.000360584,0.000354815,0.000402355,0.000372839,...,0.000389338,0.000392342,0.000365973,0.000333118,0.000360918,0.000402117,0.000339746,0.000334406,0.000337124,0.000408983,0.000350142,0.000359917,0.000390148,0.000355959,0.000338554,0.000351048,0.000358009,0.000335741,0.000329161,0.000365353,0.000409222,0.000378847,0.000375986,0.00037322,0.00037899,0.000365257,0.000389719,0.000338125,0.000369406,0.000359249,0.000334406,0.000346518,0.000320768,0.000398064,0.000329876,0.000383425,0.000360012,0.000370359,0.000361538,0.000517178
std_score_time,2.2446e-05,3.10885e-05,1.15322e-05,1.69399e-05,1.66984e-05,2.47253e-05,9.40422e-05,5.34838e-05,4.38895e-05,5.41289e-06,2.5439e-05,1.52729e-05,3.43079e-05,3.44079e-05,4.79987e-05,3.06186e-05,5.60196e-05,2.53094e-05,2.48822e-05,2.06589e-05,5.28537e-06,3.93688e-05,1.3458e-05,1.85234e-05,9.11994e-06,1.57402e-05,2.20336e-05,2.05275e-05,2.7676e-05,1.75985e-05,0.000111281,8.80485e-06,4.89996e-05,1.22217e-05,1.13249e-05,3.31014e-05,1.21529e-05,1.64653e-05,5.78041e-05,7.52043e-06,...,3.83906e-05,2.46634e-05,2.3956e-05,1.83359e-05,5.07518e-05,7.23226e-05,2.0895e-05,9.12941e-06,7.95189e-06,8.94834e-05,1.15466e-05,3.25137e-05,3.21116e-05,1.6483e-05,4.81818e-06,1.58584e-05,3.43867e-05,1.14445e-05,6.85817e-06,1.64661e-05,4.75664e-05,1.66409e-05,5.18478e-05,2.22503e-05,9.78806e-06,2.86301e-05,2.37164e-05,1.25675e-05,2.10104e-05,1.28988e-05,1.21032e-05,1.01339e-05,7.64517e-06,2.87429e-05,1.37049e-05,6.09122e-05,3.4842

In [49]:
tuned_model.best_params_

{'max_depth': 77, 'min_samples_split': 16}

In [51]:
best_model = tuned_model.best_estimator_

In [52]:
print(best_model.score(x_train_val, t_train_val))
print(best_model.score(x_test, t_test))

0.9692307692307692
0.9210526315789473


**ベイズ最適化**

In [53]:
!pip install optuna
import optuna

     |████████████████████████████████| 204kB 7.9MB/s 
     |████████████████████████████████| 1.1MB 10.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 122kB 35.6MB/s 
     |████████████████████████████████| 112kB 38.3MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159543 sha256=926c4d23df23c46697815bb2ead3e6fc8f940cff1f7e8858d5cd5ab2ca99d1a7
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-1.5.0-cp36-none-any.whl size=276145 sha256=ec09e3a0d0f6da920d22f8c30c5f37e03b45749324423d17c4caecf4e600bb28
  Stored in directory: /root/.cache/pip/wheels/38

In [54]:
from sklearn.model_selection import cross_val_score

In [56]:
def objective(trial, x, t, cv):
      # ハイパーパラメータごとに探索範囲を指定
    max_depth = trial.suggest_int('max_depth', 2, 100)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 100)

    # 学習に使用するアルゴリズムを指定
    estimator = DecisionTreeClassifier(
      max_depth = max_depth,
      min_samples_split = min_samples_split
    )

    # 学習の実行、検証結果の表示
    print('Current_params : ', trial.params)
    accuracy = cross_val_score(estimator, x, t, cv=cv).mean()
    return accuracy

In [57]:
study = optuna.create_study(direction='maximize')

In [58]:
# K 分割交差検証の K
cv = 5
# 目的関数の最適化
study.optimize(lambda trial: objective(trial, x_train_val, t_train_val, cv), n_trials=10)

print(study.best_trial)

Current_params :  {'max_depth': 58, 'min_samples_split': 29}


[I 2020-07-15 08:55:01,851] Finished trial#0 with value: 0.9120879120879121 with parameters: {'max_depth': 58, 'min_samples_split': 29}. Best is trial#0 with value: 0.9120879120879121.


Current_params :  {'max_depth': 9, 'min_samples_split': 44}


[I 2020-07-15 08:55:02,003] Finished trial#1 with value: 0.9208791208791208 with parameters: {'max_depth': 9, 'min_samples_split': 44}. Best is trial#1 with value: 0.9208791208791208.


Current_params :  {'max_depth': 85, 'min_samples_split': 69}


...

Current_params :  {'max_depth': 56, 'min_samples_split': 34}


[I 2020-07-15 08:55:03,067] Finished trial#8 with value: 0.9164835164835166 with parameters: {'max_depth': 56, 'min_samples_split': 34}. Best is trial#4 with value: 0.9208791208791209.


Current_params :  {'max_depth': 82, 'min_samples_split': 51}


[I 2020-07-15 08:55:03,218] Finished trial#9 with value: 0.9164835164835164 with parameters: {'max_depth': 82, 'min_samples_split': 51}. Best is trial#4 with value: 0.9208791208791209.


FrozenTrial(number=4, value=0.9208791208791209, datetime_start=datetime.datetime(2020, 7, 15, 8, 55, 2, 312289), datetime_complete=datetime.datetime(2020, 7, 15, 8, 55, 2, 458680), params={'max_depth': 34, 'min_samples_split': 89}, distributions={'max_depth': IntUniformDistribution(high=100, low=2, step=1), 'min_samples_split': IntUniformDistribution(high=100, low=2, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=4, state=TrialState.COMPLETE)


In [59]:
study.best_params

{'max_depth': 34, 'min_samples_split': 89}

In [60]:
# 最適なハイパーパラメータを設定したモデルの定義
best_model = DecisionTreeClassifier(**study.best_params)

# モデルの学習
best_model.fit(x_train_val, t_train_val)

# モデルの検証
print(best_model.score(x_train_val, t_train_val))
print(best_model.score(x_test, t_test))

0.9582417582417583
0.9649122807017544
